# Traffic Violation RAG System
In this exam, you will implement a Retrieval-Augmented Generation (RAG) system that uses a language model and a vector database to answer questions about traffic violations. The goal is to generate answers with relevant data based on a dataset of traffic violations and fines.

Here are helpful resources:
* [LangChain](https://www.langchain.com/)
* [groq cloud documentation](https://console.groq.com/docs/models)
* [LangChain HuggingFace](https://python.langchain.com/docs/integrations/text_embedding/sentence_transformers/)
* [Chroma Vector Store](https://python.langchain.com/docs/integrations/vectorstores/chroma/)
* [Chroma Website](https://docs.trychroma.com/getting-started)
* [ChatGroq LangChain](https://python.langchain.com/docs/integrations/chat/groq/)
* [LLM Chain](https://api.python.langchain.com/en/latest/chains/langchain.chains.llm.LLMChain.html#langchain.chains.llm.LLMChain)

Dataset [source](https://www.moi.gov.sa/wps/portal/Home/sectors/publicsecurity/traffic/contents/!ut/p/z0/04_Sj9CPykssy0xPLMnMz0vMAfIjo8ziDTxNTDwMTYy83V0CTQ0cA71d_T1djI0MXA30gxOL9L30o_ArApqSmVVYGOWoH5Wcn1eSWlGiH1FSlJiWlpmsagBlKCQWqRrkJmbmqRqUZebngB2gUJAKdERJZmqxfkG2ezgAhzhSyw!!/)

Some installs if needed:
```python
!pip install langchain_huggingface langchain langchain-community langchain_chroma Chroma langchain_groq LLMChain
```

In [1]:
!kaggle datasets download -d khaledzsa/dataset
!unzip dataset.zip

Dataset URL: https://www.kaggle.com/datasets/khaledzsa/dataset
License(s): unknown
  0% 0.00/3.73k [00:00<?, ?B/s]
100% 3.73k/3.73k [00:00<00:00, 7.82MB/s]
Archive:  dataset.zip
  inflating: Dataset.csv             


## Step 1: Install Required Libraries

To begin, install the necessary libraries for this project. The libraries include `LangChain` for building language model chains, and `Chroma` for managing a vector database.

In [2]:
!pip install langchain_huggingface langchain langchain-community langchain_chroma Chroma langchain_groq LLMChain

  Preparing metadata (setup.py) ... done
ERROR: Could not find a version that satisfies the requirement LLMChain (from versions: none)
ERROR: No matching distribution found for LLMChain


# Step 2: Load the Traffic Violations Dataset

You are provided with a dataset of traffic violations. Load the CSV file into a pandas DataFrame and preview the first few rows of the dataset using `.head()`. You can also try and see the dataset's characteristics.

In [3]:
import pandas as pd


In [4]:
df = pd.read_csv('/content/dataset.zip')
df.head()

,المخالفة,الغرامة
0,قيادة المركبة في الأسواق التي لا يسمح بالقيادة...,الغرامة المالية 100 - 150 ريال
1,ترك المركبة مفتوحة وفي وضع التشغيل بعد مغادرتها.,الغرامة المالية 100 - 150 ريال
2,عدم وجود تأمين ساري للمركبة.,الغرامة المالية 100 - 150 ريال
3,عبور المشاة للطرق من غير الأماكن المخصصة لهم.,الغرامة المالية 100 - 150 ريال
4,عدم تقيد المشاة بالإشارات الخاصة بهم.,الغرامة المالية 100 - 150 ريال


In [5]:
df.shape

(104, 2)

In [7]:
df['الغرامة'].unique()

array(['الغرامة المالية 100 - 150 ريال', 'الغرامة المالية 150 - 300 ريال',
       'الغرامة المالية 300 - 500 ريال', 'الغرامة المالية 500 - 900 ريال',
       'الغرامة المالية 1000 - 2000 ريال',
       'الغرامة المالية 3000 - 6000 ريال',
       'الغرامة المالية 5000 - 10000 ريال'], dtype=object)

## Step 3: Create Markdown Content from the Dataset

For each traffic violation in the dataset, you will generate markdown text that describes the violation and the associated fine. Create a loop to iterate through the dataset and store the generated markdown in a list. Each fine should look like this:

**المخالفة** - الغرامة

In [9]:
markdown_list = []

for index, row in df.iterrows():
    violation = row['المخالفة']
    fine = row['الغرامة']
    markdown_text = f"{violation} - {fine}"
    markdown_list.append(markdown_text)

for text in markdown_list[:5]:
    print(text)


قيادة المركبة في الأسواق التي لا يسمح بالقيادة فيها. - الغرامة المالية 100 - 150 ريال
ترك المركبة مفتوحة وفي وضع التشغيل بعد مغادرتها. - الغرامة المالية 100 - 150 ريال
عدم وجود تأمين ساري للمركبة. - الغرامة المالية 100 - 150 ريال
عبور المشاة للطرق من غير الأماكن المخصصة لهم. - الغرامة المالية 100 - 150 ريال
عدم تقيد المشاة بالإشارات الخاصة بهم. - الغرامة المالية 100 - 150 ريال


## Step 4: Chunk the Markdown Data

Using LangChain's `RecursiveCharacterTextSplitter`, split the markdown texts into smaller chunks that will be stored in the vector database.

In [13]:
!pip install langchain langchain-community langchain_huggingface langchain_chroma


  Using cached langchain-0.3.0-py3-none-any.whl.metadata (7.1 kB)
  Using cached langchain_community-0.3.0-py3-none-any.whl.metadata (2.8 kB)
  Using cached langchain_huggingface-0.1.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached langchain_chroma-0.1.4-py3-none-any.whl.metadata (1.6 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.2/599.2 kB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 66.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━

In [14]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20)

chunks = []

for text in markdown_list:
    split_texts = text_splitter.split_text(text)
    chunks.extend(split_texts)

for chunk in chunks[:5]:
    print(chunk)


قيادة المركبة في الأسواق التي لا يسمح بالقيادة فيها. - الغرامة المالية 100 - 150 ريال
ترك المركبة مفتوحة وفي وضع التشغيل بعد مغادرتها. - الغرامة المالية 100 - 150 ريال
عدم وجود تأمين ساري للمركبة. - الغرامة المالية 100 - 150 ريال
عبور المشاة للطرق من غير الأماكن المخصصة لهم. - الغرامة المالية 100 - 150 ريال
عدم تقيد المشاة بالإشارات الخاصة بهم. - الغرامة المالية 100 - 150 ريال


## Step 5: Generate Embeddings for the Documents

Generate embeddings for the chunks of text using HuggingFace's pre-trained Arabic language model. These embeddings will be stored in a `Chroma` vector store.

In [24]:
model_name = "sentence-transformers/multi-qa-mpnet-base-dot-v1"
embeddings = HuggingFaceEmbeddings(model_name=model_name)

vector_store = Chroma(embedding_function=embeddings)

vector_store.add_texts(chunks)

print(f"Number of texts stored: {len(chunks)}")

Number of texts stored: 104


# Step 6: Define the RAG Prompt Template

Define a custom prompt template in Arabic to retrieve traffic violation-related answers based on the context. Ensure the template encourages the model to give **advice** in **Arabic**, staying within the context provided.

In [25]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template="""
    لديك سياق حول مخالفات المرور والغرامات.
    يرجى قراءة السياق بعناية ثم أجب على السؤال التالي:

    السياق: {context}

    السؤال: {question}

    الإجابة (يرجى تقديم نصائح باللغة العربية):
    """
)

context_example = "عبور المشاة للطرق من غير الأماكن المخصصة لهم - الغرامة المالية 100 - 150 ريال."
question_example = "ما هي العقوبة لعبور المشاة للطرق بشكل غير قانوني؟"

prompt = prompt_template.format(context=context_example, question=question_example)
print(prompt)



    لديك سياق حول مخالفات المرور والغرامات. 
    يرجى قراءة السياق بعناية ثم أجب على السؤال التالي:

    السياق: عبور المشاة للطرق من غير الأماكن المخصصة لهم - الغرامة المالية 100 - 150 ريال.

    السؤال: ما هي العقوبة لعبور المشاة للطرق بشكل غير قانوني؟

    الإجابة (يرجى تقديم نصائح باللغة العربية):
    


## Step 7: Initialize the Language Model

In [28]:

from langchain.llms import HuggingFacePipeline

model_id = "google/flan-t5-base"
llm = HuggingFacePipeline.from_model_id(model_id=model_id, task="text2text-generation")


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Initialize the language model using the Groq API. Set up the model with a specific configuration, including the API key, temperature setting, and model name.

## Step 8: Create the LLM Chain

Now, you will create an LLM Chain that combines the language model and the prompt template you defined. This chain will be used to generate responses based on the retrieved context.

In [29]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vector_store.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt_template}
)


## Step 9: Implement the Query Function

Create a function `query_rag` that will take a user query as input, retrieve relevant context from the vector store, and use the language model to generate a response based on that context.

In [30]:

def query_rag(query):
  """
  This function takes a user query, retrieves relevant context from the vector store,
  and uses the language model to generate a response based on that context.

  Args:
      query: The user's question.

  Returns:
      A dictionary containing the generated answer and the source documents.
  """
  result = qa_chain({"query": query})
  return result


## Step 10: Inference - Running Queries in the RAG System

In this final step, you will implement an inference pipeline to handle real-time queries. You will allow the system to retrieve the most relevant violations and fines based on a user's input and generate a response.

1. Inference Workflow:

  * The user inputs a query (e.g., "ماهي الغرامة على القيادة بدون رخصة؟").
  * The system searches for the most relevant context from the traffic violation vector store.
  * It generates an answer and advice based on the context.

2. Goal:
  * Run the inference to answer questions based on the traffic violation dataset.

In [ ]:
!pip install langchain
from langchain.schema import Document

def inference_pipeline():
    while True:
        user_query = input("ادخل سؤالك حول مخالفات المرور (أو اكتب 'exit' للخروج): ")

        if user_query.lower() == 'exit':
            break

        context = vector_store.similarity_search(user_query)

        context_text = "\n".join([item.page_content for item in context])

        prompt = prompt_template.format(context=context_text, question=user_query)


        answer = llm(prompt)

        print(f"الإجابة: {answer}\n")

inference_pipeline()

ادخل سؤالك حول مخالفات المرور (أو اكتب 'exit' للخروج): ماهي الغرامة على القيادة بلا رخصة؟


<ipython-input-33-28d9c4a68ffe>:23: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  answer = llm(prompt)
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


الإجابة:          

